In [10]:
from Bio import SeqIO
from glob import glob 
from subprocess import call, check_output
from itertools import product

In [11]:
# params and paths

read_dir = 'sanger_reads/'
my_sample_names = [ 'Alex{}-{}'.format( i, j ) for i, j in product( range( 1, 13 ), range( 1, 4 ) ) ]
wt_pep = 'order_peptide_sequences.fasta' 

In [12]:
# convert all the reads to FASTQ

for read in glob( '{}/*ab1'.format( read_dir ) ):
    SeqIO.convert( read, 'abi', read.replace( 'ab1', 'fastq' ), 'fastq' )

In [22]:
# pair and merge the reads 

expect = 0
for sample_name in my_sample_names:
    fwd = glob( '{}/*{}_*T7.fastq'.format( read_dir, sample_name ) ) 
    rev = glob( '{}/*{}_*T7-Ter.fastq'.format( read_dir, sample_name ) )
    if len( fwd ) == len( rev ) == 1:
        my_sample = '{}/{}'.format( read_dir, sample_name )
        cmd = [ 'pear', '-f', fwd[0], '-r', rev[0], '-o', my_sample ]
        call( cmd )
        expect += 1 

In [23]:
# convert all the assemblies to FASTA

for assembly in glob( '{}/*.assembled.fastq'.format( read_dir ) ):
    SeqIO.convert( assembly, 'fastq', assembly.replace( 'fastq', 'fasta' ), 'fasta' )

In [24]:
! cat sanger_reads/*assembled.fasta > my_reads.fa 
! blastn -subject order_nucleotide_sequences.fasta -query my_reads.fa
! blastx -subject order_peptide_sequences.fasta -query my_reads.fa

BLASTN 2.2.31+


Query= U0619BF170-36_Alex12-3_T7

Length=784

Subject= 1bvv_A

Length=623


***** No hits found *****



Lambda      K        H
    1.33    0.621     1.12 

Gapped
Lambda      K        H
    1.28    0.460    0.850 

Effective search space used: 471692


Query= U0619BF170-36_Alex12-3_T7

Length=784

Subject= 1bvv_C

Length=623


***** No hits found *****



Lambda      K        H
    1.33    0.621     1.12 

Gapped
Lambda      K        H
    1.28    0.460    0.850 

Effective search space used: 471692


Query= U0619BF170-36_Alex12-3_T7

Length=784

Subject= 1fcy_A

Length=776


 Score = 1323 bits (716),  Expect = 0.0
 Identities = 734/743 (99%), Gaps = 0/743 (0%)
 Strand=Plus/Plus

Query  1    TACATATGGCAAGTCCGCAGCTGGAAGAACTGATTTACAAAATTAGCAAAACACCCCAAG  60
            |||||||||||||||||||||||||||||||||||  |||| |||||||| ||| |||||
Sbjct  34   TACATATGGCAAGTCCGCAGCTGGAAGAACTGATTACCAAAGTTAGCAAAGCACACCAAG  93

Query  61   AAACCTTTCCGAACCTGTGTCAGCTGGGTAAATTTTCCCCCAATAGCAGCGCA

### Sequencing results  
```
Could be: 

U0619BF170-36_Alex12-3_T7, 1fcy_A *best guess 
U0619BF170-36_Alex12-3_T7, 1fcy_C
U0619BF170-36_Alex12-3_T7, 1fcy_C_3Tyr

* explanation: shitty beginning of fwd read 

Actually is:

U0619BF170-21_Alex7-3_T7, 1upv_A
```